In [8]:
import pygad
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
import copy
import random
import math
import time

from Modules.Helper import Helper
from Modules.Solvers import Solvers
from Modules.Plotters import Plotters
from Modules.Equation import Equation

In [9]:
# Inicialização dos parêmetros
labels = ['A', 'B', 'C', 'D', 'E']
df, max_data = Helper.load_data(filename='Data/GRN5_DATA.txt', labels=labels)
initial_conditions = np.array([df[label].iloc[0] for label in labels])
t_span = (df['t'].iloc[0], df['t'].iloc[-1])
t_eval = np.array(df['t'])
original = np.array(df[labels]).T

In [10]:
# Definindo os limites para os coeficientes
bounds = {
    'tau': (0.1, 10.0),
    'k': (0.1, 5.0),
    'n': (0.1, 5.0),
}

# Tamanho do indivíduo = número de coeficientes
IND_SIZE = 19

In [11]:
# Classe dos coeficientes
class Coefficient:
    def __init__(self, bounds):
        self.val = random.uniform(*bounds)
        self.bounds = bounds
    
    def __repr__(self):
        return f"value={self.val}"

In [21]:
# Classe do indivíduo/solução candidata
class Individual:
    def __init__(self):
        self.coeffs = {
            'A': {
                'E': {'n': None, 'k': None, '-': True},
                'tau': None
            },
            'B': {
                'A': {'n': None, 'k': None, '-': False},
                'tau': None
            },
            'C': {
                'B': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'D': {
                'C': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'E': {
                'D': {'n': None, 'k': None, '-': False},
                'B': {'n': None, 'k': None, '-': False},
                'E': {'n': None, 'k': None, '-': False},
                'tau': None,
            }
        }
        self.ind_size = IND_SIZE
        self.fitness = np.inf
       
    # Equações do Modelo GRN5
    @staticmethod
    def system(t, y, equation):
        vals = [Solvers.norm_hardcoded(val, max_data[label]) for val, label in zip(y, labels)]
        N_A, N_B, N_C, N_D, N_E = vals
        
        dA = equation.full_eq(vals, 'A', 'E')
        dB = equation.full_eq(vals, 'B', 'A')
        dC = equation.full_eq(vals, 'C', 'B')
        dD = equation.full_eq(vals, 'D', 'C')
        dE = equation.complex_eqs(vals, 'E', [['+B', '+D'], ['+D', '+E']])

        return [dA, dB, dC, dD, dE]
    
    # Cálculo do Erro Absoluto
    @staticmethod
    def absolute_error(original, predicted):
        return sum(sum(abs(original-predicted)))
    
    # Cálculo do Erro pela distância entre pontos
    @staticmethod
    def squared_error(original, predicted):
        return sum(sum( (original-predicted)**2 ))**(1/2)
    
    def calculate_fitness(self, method='RK45'):
        equation = Equation(self.numerical_coeffs, labels)
        y = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation, )).y
        self.fitness = self.absolute_error(original, y)
            
    @staticmethod
    def initialize_ind(bounds):
        ind = Individual()
        for key, label in ind.coeffs.items():
            label['tau'] = Coefficient(bounds['tau'])
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = Coefficient(bounds['n'])
                    coeffs['k'] = Coefficient(bounds['k'])
                    
        ind.calculate_fitness()
        return ind
    
    @property
    def numerical_coeffs(self):
        
        numerical_coeffs = copy.deepcopy(self.coeffs)
        for key, label in numerical_coeffs.items():
            label['tau'] = label['tau'].val
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = int(coeffs['n'].val)
                    coeffs['k'] = coeffs['k'].val
                    
        return numerical_coeffs
    
    
    def plot(self, method='RK45'):
        methods = [method]
        results = {}
        equation = Equation(self.numerical_coeffs, labels)
        results[method] = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation)).y
        Plotters.plot_methods(results=results,t=t_eval, methods=methods, labels=labels)
        Plotters.plot_comparison(results=results, t=t_eval, df=df, methods=methods, labels=labels)
        
        
    def __repr__(self):
        coeffs_repr = {k: v for k, v in self.coeffs.items()}
        return f"Individual(fitness={self.fitness}, coeffs={coeffs_repr}, ind_size={self.ind_size})"